## IMPORTAR LAS LIBRERIAS

Actualizar a las que se usen finalmente en tu proyecto.

In [1]:
import os
import numpy as np
import pandas as pd
import pickle  

#Automcompletar rápido
%config IPCompleter.greedy=True

#Desactivar la notación científica
pd.options.display.float_format = '{:.2f}'.format

#Desactivar los warnings
import warnings
warnings.filterwarnings("ignore")

#Mostrar el máximo de filas posibles de una tabla
pd.set_option('display.max_rows', 100) #Número de filas que deben verse. None = Máx

#Mostrar mas caracteres de las columnas. Se usa cuando se corta el texto
pd.set_option('display.max_colwidth', None) #Número de caractres que deben verse. None = Máx

#Representación visual de un pipeline
from sklearn import set_config
set_config(display = 'diagram') #diagram/text 

#Transformación de variables
from janitor import clean_names
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import MinMaxScaler

#Modelos ML: CLASIFICACIÓN
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
#Métrica de error: Habrá que incluir la que necesitemos
from sklearn.metrics import roc_auc_score

#Modelos ML: REGRESIÓN
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingRegressor
#Métrica de error: Habrá que incluir la que necesitemos
from sklearn.metrics import mean_absolute_error


## CARGAR LOS DATOS

### Ruta del proyecto

In [2]:
ruta_proyecto = 'C:/Users/Oscar/OneDrive - FM4/Escritorio/Python Data Mastery/EstructuraDirectorio/03_MACHINE_LEARNING/08_CASOS/003_RIESGOS'

### Nombre del fichero de datos

In [3]:
nombre_fichero_datos = 'prestamos.csv'

### Cargar los datos

In [4]:
ruta_completa = ruta_proyecto + '/02_Datos/01_Originales/' + nombre_fichero_datos

df = pd.read_csv(ruta_completa,index_col=0)

### Seleccionar solo las variables finales

#### Apuntar (manualmente) la lista de variables finales sin extensiones

In [5]:
#Metemos todas las variables usadas con sus nombres originales. Algunas de las variables incluidas las hemos borrado en pasos
#posteriores pero, esas variables, las hemos usado para hacer transformaciones, así que hay que incluirlas. Son:'estado', 
#'imp_amortizado', 'imp_recuperado'

variables_finales = ['antigüedad_empleo',
                     'ingresos_verificados',
                     'rating',
                     'vivienda',
                     'finalidad',
                     'num_cuotas',
                     'ingresos',
                     'dti',
                     'num_hipotecas',
                     'num_lineas_credito',
                     'porc_tarjetas_75p',
                     'porc_uso_revolving',
                     'num_cancelaciones_12meses',
                     'num_derogatorios',
                     'num_meses_desde_ult_retraso',
                     'principal',
                     'tipo_interes',
                     'imp_cuota',
                     'imp_amortizado',
                     'imp_recuperado',
                     'estado'
                    ]

#### Crear la matriz de variables procesos (excel)

Ir a la plantilla de Excel "Fase Producción Plantilla Procesos" y crear la matriz de variables por procesos.

#### Actualizar las importaciones

Ir arriba a la celda de importacion de paquetes y actualizarlos con los que finalmente vamos a usar.

## ESTRUCTURA DE LOS DATASETS

### Eliminar registros

#### Por atípicos

In [6]:
a_eliminar = df.loc[df.ingresos > 300000].index.values

In [7]:
df = df[~df.index.isin(a_eliminar)]

### Seleccionar variables

Quedarse solo con las de la lista.

In [8]:
df = df[variables_finales]

## CREAR EL PIPELINE

### Instanciar calidad de datos

#### Crear la función

In [9]:
def calidad_datos(temp):
    # Limpiar nombres de columnas
    temp = clean_names(temp)
    
    # Rellenar nulos de columna 'antiguedad_empleo'
    temp['antiguedad_empleo'] = temp['antiguedad_empleo'].fillna('desconocido')
    
    # Eliminar dato extraño
    temp.drop(index=16130949)
   
    # Subfunción para categorizar 'porc_tarjetas_75p'
    def categorizar_tarjetas(porc):
        if porc <= 10:
            return '<10%'
        elif porc <= 75:
            return '11-75%'
        else:
            return '>75%'
    
    # Crear nueva columna categorizada
    if 'porc_tarjetas_75p' in temp.columns:
        temp['cat_tarjetas_75p'] = temp['porc_tarjetas_75p'].apply(categorizar_tarjetas)
    
    # Sustituimos por 0 los créditos que tienen los estados de impago y por 999 el resto que no han registrado nunca impagos
    temp['num_meses_desde_ult_retraso'] = np.where((temp.num_meses_desde_ult_retraso.isna())
                                               &(temp.estado.isin(['Charged Off',
                                                                   'Does not meet the credit policy. Status:Charged Off',
                                                                   'Default'])),
                                               temp['num_meses_desde_ult_retraso'].fillna(0),
                                               temp['num_meses_desde_ult_retraso'].fillna(999))
    
    # Rellenar nulos en el resto de columnas numéricas con 0
    for column in temp.select_dtypes('number').columns:
        temp[column] = temp[column].fillna(0)
    
    #Imputar por desviación típica
    def atipicos_desv_tip(variable, num_desv_tip = 4):
        #sacamos los nulos por ahora
        variable = variable.dropna()
        #calculamos los límites
        media = np.mean(variable)
        sd = np.std(variable)
        umbral = sd * num_desv_tip
        lim_inf = media - umbral
        lim_sup = media + umbral
        #encontramos los índices de los que están fuera de los límites
        indices = [indice for indice,valor in variable.items() if valor < lim_inf or valor > lim_sup]
        return(indices)
    var_atipicos_desv_tip = ['ingresos','num_hipotecas','principal','imp_cuota','imp_recuperado']
    
    #Winsorización manual
    temp['porc_uso_revolving'] = temp['porc_uso_revolving'].clip(0, 100)
    temp['dti'] = temp['dti'].clip(0, 100)    

    return temp

### Instanciar creación de variables

Dado que la creación de variables es diferente para los 3 modelos necesitamos construir 3 funciones.

#### Crear las funciones

In [10]:
def creacion_variables_pd(df):
    
    temp = df.copy()
    
    temp['target_pd'] = np.where(temp.estado.isin(['Charged Off','Does not meet the credit policy. Status:Charged Off','Default']), 1, 0)
    
    temp.vivienda = temp.vivienda.replace(['ANY','NONE','OTHER'],'MORTGAGE')
    
    temp.finalidad = temp.finalidad.replace(['wedding','educational','renewable_energy'],'otros')
    
    #Eliminamos las variables que ya no usaremos
    #'estado' la hemos usado para generar la target por lo que no podemos usarla para predecir
    #'imp_amortizado','imp_recuperado' las eliminamos porque no las tendremos cuando queramos hacer una predicción de riesgos
    #en el futuro porque no tendrán datos, por lo que no tiene sentido que las usemos para predecir ahora que están informadas
    temp.drop(columns = ['estado','imp_amortizado','imp_recuperado'],inplace = True)
    
    #Separamos entre predictoras y target
    temp_x = temp.iloc[:,:-1]
    temp_y = temp.iloc[:,-1]
    
    return(temp_x,temp_y)

In [11]:
def creacion_variables_ead(df):
    
    temp = df.copy()
    
    temp['pendiente'] = temp.principal - temp.imp_amortizado
    
    temp['target_ead'] = temp.pendiente / temp.principal
    
    temp.vivienda = temp.vivienda.replace(['ANY','NONE','OTHER'],'MORTGAGE')
    
    temp.finalidad = temp.finalidad.replace(['wedding','educational','renewable_energy'],'otros')
    
    #Eliminamos las variables que ya no usaremos
    #'estado' la hemos usado para generar la target por lo que no podemos usarla para predecir
    #'imp_amortizado','imp_recuperado' las eliminamos porque no las tendremos cuando queramos hacer una predicción de riesgos
    #en el futuro porque no tendrán datos, por lo que no tiene sentido que las usemos para predecir ahora que están informadas
    temp.drop(columns = ['estado','imp_amortizado','imp_recuperado','pendiente'],inplace = True)
    
    #Separamos entre predictoras y target
    temp_x = temp.iloc[:,:-1]
    temp_y = temp.iloc[:,-1]
    
    return(temp_x,temp_y)

In [12]:
def creacion_variables_lgd(df):
    
    temp = df.copy()
    
    temp['pendiente'] = temp.principal - temp.imp_amortizado
    
    temp['target_lgd'] = 1 - (temp.imp_recuperado / temp.pendiente)
    
    temp['target_lgd'].fillna(0,inplace=True)
    
    temp.vivienda = temp.vivienda.replace(['ANY','NONE','OTHER'],'MORTGAGE')
    
    temp.finalidad = temp.finalidad.replace(['wedding','educational','renewable_energy'],'otros')
    
    #Eliminamos las variables que ya no usaremos
    #'estado' la hemos usado para generar la target por lo que no podemos usarla para predecir
    #'imp_amortizado','imp_recuperado' las eliminamos porque no las tendremos cuando queramos hacer una predicción de riesgos
    #en el futuro porque no tendrán datos, por lo que no tiene sentido que las usemos para predecir ahora que están informadas
    temp.drop(columns = ['estado','imp_amortizado','imp_recuperado','pendiente'],inplace = True)
    
    #Separamos entre predictoras y target
    temp_x = temp.iloc[:,:-1]
    temp_y = temp.iloc[:,-1]
    
    return(temp_x,temp_y)

#### Crear los dataframes de X e y

In [26]:
x_pd, y_pd = creacion_variables_pd(calidad_datos(df))

x_ead, y_ead = creacion_variables_ead(calidad_datos(df))

x_lgd, y_lgd = creacion_variables_lgd(calidad_datos(df))

In [27]:
x_ead

,antiguedad_empleo,ingresos_verificados,rating,vivienda,finalidad,num_cuotas,ingresos,dti,num_hipotecas,num_lineas_credito,porc_tarjetas_75p,porc_uso_revolving,num_cancelaciones_12meses,num_derogatorios,num_meses_desde_ult_retraso,principal,tipo_interes,imp_cuota,cat_tarjetas_75p,pendiente
id_cliente,,,,,,,,,,,,,,,,,,,,
137387967,3 years,Source Verified,A,MORTGAGE,debt_consolidation,36 months,54000.00,19.31,2.00,10.00,33.30,45.20,0.00,0.00,10.00,15000.00,7.21,464.60,11-75%,12330.94
4798121,10+ years,Not Verified,D,RENT,debt_consolidation,36 months,65000.00,25.40,1.00,15.00,85.70,86.20,0.00,0.00,0.00,10000.00,17.77,360.38,>75%,3637.04
46641215,5 years,Verified,A,RENT,debt_consolidation,36 months,135000.00,14.68,0.00,19.00,0.00,16.30,0.00,0.00,999.00,24000.00,6.39,734.38,<10%,0.00
87998444,9 years,Source Verified,B,MORTGAGE,credit_card,60 months,188000.00,11.69,3.00,15.00,0.00,3.30,0.00,0.00,999.00,27000.00,8.99,560.35,<10%,14557.00
132883631,7 years,Source Verified,B,MORTGAGE,debt_consolidation,36 months,125000.00,9.00,1.00,6.00,33.30,69.90,0.00,0.00,16.00,22000.00,10.90,719.22,11-75%,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51876926,10+ years,Not Verified,C,MORTGAGE,debt_consolidation,36 months,42000.00,20.85,6.00,9.00,100.00,81.60,0.00,0.00,13.00,8000.00,12.29,266.83,>75%,0.00
121031962,6 years,Verified,B,MORTGAGE,other,36 months,111697.00,16.63,2.00,10.00,50.00,26.30,0.00,1.00,6.00,10000.00,9.44,320.05,11-75%,5611.49
135641397,10+ years,Source Verified,D,MORTGAGE,small_business,36 months,285000.00,6.02,3.00,9.00,25.00,21.10,0.00,0.00,25.00,30000.00,17.47,1076.62,11-75%,24612.47


### Instanciar transformación de variables

In [28]:
#ONE HOT ENCODING
var_ohe = [ 'ingresos_verificados', 'vivienda','finalidad','num_cuotas']
ohe = OneHotEncoder(sparse_output = False, handle_unknown='ignore')


#ORDINAL ENCODING
var_oe = ['antiguedad_empleo','rating','cat_tarjetas_75p']

orden_antiguedad_empleo = ['desconocido','< 1 year','1 year','2 years','3 years','4 years',
                           '5 years','6 years','7 years','8 years','9 years','10+ years']

orden_rating = ['A','B','C','D','E','F','G']

orden_cat_tarjetas_75p = ['<10%','11-75%','>75%']

oe = OrdinalEncoder(categories = [orden_antiguedad_empleo,orden_rating,orden_cat_tarjetas_75p],
                    handle_unknown = 'use_encoded_value',
                    unknown_value = 12)

#BINNING
var_bin = ['num_derogatorios']
bina = Binarizer(threshold=0)


#MIN-MAX SCALING
var_mms = ['ingresos','dti','num_lineas_credito','porc_uso_revolving',
            'principal','tipo_interes','imp_cuota']
mms = MinMaxScaler()


### Crear el pipe del preprocesamiento

#### Crear el column transformer

In [29]:
ct = make_column_transformer(
    (ohe, var_ohe),
    (oe, var_oe),
#    (te, var_te),
#    (disc_ds, var_disc_ds),
#    (disc_cs, var_disc_cs),
    (bina, var_bin),
#    (yeo, var_yeo),    
#    (qt, var_qt),
    (mms, var_mms),
#    (ss, var_ss),
#    (rs, var_rs),
    remainder='passthrough') #En este caso no hay mas variables porque las hemos usado todas. Podría ser: 'drop' o 'passthrough'

### Instanciar los modelos

#### Instanciar los algoritmos

In [30]:
modelo_pd = LogisticRegression(solver = 'saga', n_jobs=-1, C = 0.25, penalty = 'l1')

modelo_ead = HistGradientBoostingRegressor(learning_rate = 0.1,
                                          max_iter = 200,
                                          max_depth = 10,
                                          min_samples_leaf = 100,
                                          scoring = 'neg_mean_absolute_percentage_error',
                                          l2_regularization = 0.75)

modelo_lgd = HistGradientBoostingRegressor(learning_rate = 0.1,
                                          max_iter = 200,
                                          max_depth = 20,
                                          min_samples_leaf = 100,
                                          scoring = 'neg_mean_absolute_percentage_error',
                                          l2_regularization = 0)

#### Crear los pipes finales de entrenamiento

In [31]:
pipe_entrenamiento_pd = make_pipeline(ct,modelo_pd)

pipe_entrenamiento_ead = make_pipeline(ct,modelo_ead)

pipe_entrenamiento_lgd = make_pipeline(ct,modelo_lgd)

#### Guardar el pipe final de entrenamiento

Guardamos este código como objeto para poder usarlo en el reentrenamiento futuro

In [32]:
ruta_pipe_entrenamiento_pd = ruta_proyecto + '/04_Modelos/pipe_entrenamiento_pd.pickle'

with open(ruta_pipe_entrenamiento_pd, mode='wb') as file:
   pickle.dump(pipe_entrenamiento_pd, file)

In [33]:
ruta_pipe_entrenamiento_ead = ruta_proyecto + '/04_Modelos/pipe_entrenamiento_ead.pickle'

with open(ruta_pipe_entrenamiento_ead, mode='wb') as file:
   pickle.dump(pipe_entrenamiento_ead, file)

In [34]:
ruta_pipe_entrenamiento_lgd = ruta_proyecto + '/04_Modelos/pipe_entrenamiento_lgd.pickle'

with open(ruta_pipe_entrenamiento_lgd, mode='wb') as file:
   pickle.dump(pipe_entrenamiento_lgd, file)

#### Entrenar los pipes

In [35]:
pipe_ejecucion_pd = pipe_entrenamiento_pd.fit(x_pd,y_pd)
pipe_ejecucion_ead = pipe_entrenamiento_ead.fit(x_ead,y_ead)
pipe_ejecucion_lgd = pipe_entrenamiento_lgd.fit(x_lgd,y_lgd)

## GUARDAR EL PIPE

### Guardar el pipe final de ejecución

In [36]:
ruta_pipe_ejecucion_pd = ruta_proyecto + '/04_Modelos/pipe_ejecucion_pd.pickle'

with open(ruta_pipe_ejecucion_pd, mode='wb') as file:
   pickle.dump(pipe_ejecucion_pd, file)

In [37]:
ruta_pipe_ejecucion_ead = ruta_proyecto + '/04_Modelos/pipe_ejecucion_ead.pickle'

with open(ruta_pipe_ejecucion_ead, mode='wb') as file:
   pickle.dump(pipe_ejecucion_ead, file)

In [38]:
ruta_pipe_ejecucion_lgd = ruta_proyecto + '/04_Modelos/pipe_ejecucion_lgd.pickle'

with open(ruta_pipe_ejecucion_lgd, mode='wb') as file:
   pickle.dump(pipe_ejecucion_lgd, file)